In [1]:
import sys
sys.path.append("..\src")

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime

from sgcarmart_webscraper_functions import * # Imports all defined webscraping functions

# WebScraping

In [3]:
# Creating a list of main pages to iterate through
main_page_listing_list = [] # creating list to store search pages of 100 car listings
for idx, link in enumerate(range(30)):
    url = "https://www.sgcarmart.com/used_cars/listing.php?BRSR=" + str(idx * 100) + "&RPG=100&AVL=2&VEH=2"
    main_page_listing_list.append(url)

In [4]:
print(main_page_listing_list,'\n','\n', len(main_page_listing_list))

['https://www.sgcarmart.com/used_cars/listing.php?BRSR=0&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=200&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=300&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=400&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=500&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=600&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=700&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=800&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=900&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1000&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/li

## Retrieving individual listing urls from search pages of 100 listings

In [5]:
# Base url, or you can think of this as the individual car listing prefix
base_url = 'https://www.sgcarmart.com/used_cars/'
listing_urls = []

# Acquiring indvidual car listings    
for main_link in main_page_listing_list:
   
    # Make a request to the website and get the object
    content = requests.get(main_link)

    # Parse the HTML text
    soup = BeautifulSoup(content.text, 'lxml')

    # Find every single URL in the webpage , refer to this post: # https://stackoverflow.com/questions/46490626/getting-all-links-from-a-page-beautiful-soup
    # This returns a list of every tag that contains a link in one main link (each element in main page listing)
    links = soup.find_all('a')
    
    # Create a list for storing all the individual listing urls
    
    for link in links:
        # Get link in <a href>
        suffix = link.get('href')

        # Check if 'ID=' and 'DL=' exist in the string
        if ('ID=' in suffix) and ('DL=' in suffix):

            # Concatenate the two strings if they do
            listing_url = base_url + suffix
            
            # Append result to the list
            listing_urls.append(listing_url)
            
#     Removing duplicates
    set_listing_urls = set(listing_urls)
    listing_urls = list(set_listing_urls)
    
    # Prevent oneself from getting blocked from the website
    time.sleep(1)

In [9]:
print(len(listing_urls))
print(len(set(listing_urls)))
print(len(list(set(listing_urls))))

3000
3000
3000


In [10]:
print(listing_urls[:10])

['https://www.sgcarmart.com/used_cars/info.php?ID=852773&DL=2792', 'https://www.sgcarmart.com/used_cars/info.php?ID=853522&DL=2191', 'https://www.sgcarmart.com/used_cars/info.php?ID=862824&DL=3495', 'https://www.sgcarmart.com/used_cars/info.php?ID=850975&DL=3359', 'https://www.sgcarmart.com/used_cars/info.php?ID=864258&DL=2077', 'https://www.sgcarmart.com/used_cars/info.php?ID=751129&DL=2296', 'https://www.sgcarmart.com/used_cars/info.php?ID=822459&DL=1344', 'https://www.sgcarmart.com/used_cars/info.php?ID=845338&DL=2792', 'https://www.sgcarmart.com/used_cars/info.php?ID=842017&DL=3475', 'https://www.sgcarmart.com/used_cars/info.php?ID=861127&DL=1091']


## DataFrame Creation

In [11]:
# Creating an empty DataFrame for attributes of interest
df = pd.DataFrame(columns=['LISTING_URL', 'BRAND', 'PRICE', 'DEPRE_VALUE_PER_YEAR',
       'REG_DATE', 'MILEAGE_KM', 'MANUFACTURED_YEAR',
       'ROAD_TAX_PER_YEAR','TRANSMISSION', 'DEREG_VALUE_FROM_SCRAPE_DATE',
       'SCRAPE_DATE', 'OMV', 'ARF', 'COE_FROM_SCRAPE_DATE',
       'DAYS_OF_COE_LEFT', 'ENGINE_CAPACITY_CC', 'CURB_WEIGHT_KG',
       'NO_OF_OWNERS', 'VEHICLE_TYPE'])

In [14]:
filename = 'sgcarmart_used_cars_prices3'
i = 0 # Indexing rows in the DF

for listingurl in listing_urls:
    response = requests.get(listingurl)
    listing_url = BeautifulSoup(response.text, 'lxml')
    
    # Retrieval functions to pull data from the Individual Listings after they have been parsed
    df.loc[i, 'LISTING_URL'] = listingurl
    df.loc[i, 'BRAND'] = brand_retrieval(listing_url)
    df.loc[i, 'PRICE'] = price_retrieval(listing_url)
    try:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = depreciation_value_per_year_retrieval(listing_url)
    except:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'REG_DATE'] = registered_date_retrieval(listing_url)
    except:
        df.loc[i, 'REG_DATE'] = np.nan
    
    try:
        df.loc[i, 'MILEAGE_KM'] = mileage_retrieval(listing_url)
    except:
        df.loc[i, 'MILEAGE_KM'] = np.nan

    try:
        df.loc[i, 'MANUFACTURED_YEAR'] = manufactured_year_retrieval(listing_url)
    except: 
        df.loc[i, 'MANUFACTURED_YEAR'] = np.nan
    
    try:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = road_tax_retrieval(listing_url)
    except:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'TRANSMISSION'] = transmission_retrieval(listing_url)
    except:
        df.loc[i, 'TRANSMISSION'] = np.nan

        
    try:
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = dereg_value_retrieval(listing_url)
    except: 
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = np.nan
        
    df.loc[i, 'SCRAPE_DATE'] = datetime.now().strftime("%d/%m/%Y")
    
    try:
        df.loc[i, 'OMV'] = omv_retrieval(listing_url)
    except: 
        df.loc[i, 'OMV'] = np.nan

    try:
        df.loc[i, 'ARF'] = arf_retrieval(listing_url)
    except: 
        df.loc[i, 'ARF'] = np.nan
        
    try:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = coe_retrieval(listing_url)
    except:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = np.nan
        
    try:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = days_of_coe_retrieval(listing_url)
    except:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = np.nan
        
    try:
        df.loc[i, 'ENGINE_CAPACITY_CC'] = engine_capacity_retrieval(listing_url)
    except: 
        df.loc[i, 'ENGINE_CAPACITY_CC'] = np.nan
        
    try:
        df.loc[i, 'CURB_WEIGHT_KG'] = curb_weight_retrieval(listing_url)
    except:
        df.loc[i, 'CURB_WEIGHT_KG'] = np.nan
        
    try:
        df.loc[i, 'NO_OF_OWNERS'] = number_of_owners_retrieval(listing_url)
    except:
        df.loc[i, 'NO_OF_OWNERS'] = np.nan
        
    try:
        df.loc[i, 'VEHICLE_TYPE'] = type_of_vehicle_retrieval(listing_url)
    except:
        df.loc[i, 'VEHICLE_TYPE'] = np.nan
        
    df.to_csv("{}.csv".format(filename))    
        
    i += 1 # Allows next car listing to be put into a next row in the dataframe
    time.sleep(1)  # Prevents us from getting locked out of the website
    

In [15]:
df = pd.read_csv('sgcarmart_used_cars_prices3.csv',index_col=0)
df

,LISTING_URL,BRAND,PRICE,DEPRE_VALUE_PER_YEAR,REG_DATE,MILEAGE_KM,MANUFACTURED_YEAR,ROAD_TAX_PER_YEAR,TRANSMISSION,DEREG_VALUE_FROM_SCRAPE_DATE,SCRAPE_DATE,OMV,ARF,COE_FROM_SCRAPE_DATE,DAYS_OF_COE_LEFT,ENGINE_CAPACITY_CC,CURB_WEIGHT_KG,NO_OF_OWNERS,VEHICLE_TYPE
0,https://www.sgcarmart.com/used_cars/info.php?I...,Hyundai,26888,5470,03-Oct-2009,137991,2009,738,Auto,14484,03/11/2019,11605,11605,14743,1467,1591,1264,2,Mid-Sized Sedan
1,https://www.sgcarmart.com/used_cars/info.php?I...,MINI,98800,11820,08-Jun-2017,39900,2017,684,Auto,54050,03/11/2019,25768,18076,53300,2769,1499,1175,1,Hatchback
